# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
# import graphlab

import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
# sales = graphlab.SFrame('kc_house_data.gl/')

sales = pd.read_csv('kc_house_data.csv', 
                    dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
# train_data,test_data = sales.random_split(.8,seed=0)

train_data = pd.read_csv('kc_house_train_data.csv', 
                         dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

test_data = pd.read_csv('kc_house_test_data.csv', 
                         dtype = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int})

# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [4]:
# example_features = ['sqft_living', 'bedrooms', 'bathrooms']
# example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
#                                                  validation_set = None)

reg = LinearRegression()
reg.fit(train_data.loc[:, ['sqft_living', 'bedrooms', 'bathrooms']], 
        train_data.loc[:, ['price']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [5]:
# example_weight_summary = example_model.get("coefficients")
# print example_weight_summary

print('Coefficients: \n', reg.coef_)

Coefficients: 
 [[   315.40669062 -65081.88711588   6942.16598637]]


# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [6]:
# example_predictions = example_model.predict(train_data)
# print example_predictions[0] # should be 271789.505878

example_predictions = reg.predict(train_data.loc[:, ['sqft_living', 'bedrooms', 'bathrooms']])
example_predictions[0]

array([271789.26537997])

# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [7]:
def get_residual_sum_of_squares(model, data, outcome):
    """
    type:
        model: LinearRegression object
        data: features, a pandas dataframe
        outcome: expected response, a ndarray
    rtype:
        RSS, a scalar
    """
    # First get the predictions
    pred = np.ravel(model.predict(data))

    # Then compute the residuals/errors
    res = outcome - pred

    # Then square and add them up
    RSS = np.sum(res ** 2)

    return(RSS)

Test your function by computing the RSS on TEST data for the example model:

In [8]:
# rss_example_train = get_residual_sum_of_squares(example_model, test_data, test_data['price'])
# print rss_example_train # should be 2.7376153833e+14

rss_example_train = get_residual_sum_of_squares(reg, 
                                                test_data.loc[:, ['sqft_living', 'bedrooms', 'bathrooms']], 
                                                test_data.loc[:, 'price'].values)
print('%E' % rss_example_train) # should be 2.7376153833e+14
print(2.7376153833e+14 / 2.737619E+14 - 1)

2.737619E+14
-1.3211115206246404e-06


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [9]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [10]:
# train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
# test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

train_data = train_data.assign(bedrooms_squared = train_data['bedrooms'] ** 2)
test_data = test_data.assign(bedrooms_squared = test_data['bedrooms'] ** 2)

In [11]:
# create the remaining 3 features in both TEST and TRAIN data

train_data = train_data.assign(bed_bath_rooms = train_data['bedrooms'] * train_data['bathrooms'])
test_data = test_data.assign(bed_bath_rooms = test_data['bedrooms'] * test_data['bathrooms'])

train_data = train_data.assign(log_sqft_living = np.log(train_data['sqft_living']))
test_data = test_data.assign(log_sqft_living = np.log(test_data['sqft_living']))

train_data = train_data.assign(lat_plus_long = train_data['lat'] + train_data['long'])
test_data = test_data.assign(lat_plus_long = test_data['lat'] + test_data['long'])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [22]:
print('mean of bedrooms_squared: %.2f' % np.mean(test_data['bedrooms_squared']))
print('mean of bed_bath_rooms: %.2f' % np.mean(test_data['bed_bath_rooms']))
print('mean of log_sqft_living: %.2f' % np.mean(test_data['log_sqft_living']))
print('mean of lat_plus_long: %.2f' % np.mean(test_data['lat_plus_long']))

mean of bedrooms_squared: 12.45
mean of bed_bath_rooms: 7.50
mean of log_sqft_living: 7.55
mean of lat_plus_long: -74.65


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [13]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [14]:
# Learn the three models: (don't forget to set validation_set = None)
reg_1 = LinearRegression()
reg_1.fit(train_data.loc[:, model_1_features], 
          train_data.loc[:, ['price']])

reg_2 = LinearRegression()
reg_2.fit(train_data.loc[:, model_2_features], 
          train_data.loc[:, ['price']])

reg_3 = LinearRegression()
reg_3.fit(train_data.loc[:, model_3_features], 
          train_data.loc[:, ['price']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [15]:
# Examine/extract each model's coefficients:
print('Coefficients of model 1: \n', reg_1.coef_)
print('Coefficients of model 2: \n', reg_2.coef_)
print('Coefficients of model 3: \n', reg_3.coef_)

Coefficients of model 1: 
 [[ 3.12258646e+02 -5.95865332e+04  1.57067421e+04  6.58619264e+05
  -3.09374351e+05]]
Coefficients of model 2: 
 [[ 3.06610053e+02 -1.13446368e+05 -7.14613083e+04  6.54844630e+05
  -2.94298969e+05  2.55796520e+04]]
Coefficients of model 3: 
 [[ 5.29422820e+02  3.45142296e+04  6.70607813e+04  5.34085611e+05
  -4.06750711e+05 -8.57050439e+03 -6.78858667e+03 -5.61831484e+05
   1.27334900e+05]]


**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?**

Answer: positive.

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?**

Answer: negative.

Think about what this means.

Answer: **\[pending\]**

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [19]:
# Compute the RSS on TRAINING data for each of the three models and record the values:
rss_1_train = get_residual_sum_of_squares(reg_1,
                                          train_data.loc[:, model_1_features],
                                          train_data.loc[:, 'price'].values)

rss_2_train = get_residual_sum_of_squares(reg_2,
                                          train_data.loc[:, model_2_features],
                                          train_data.loc[:, 'price'].values)

rss_3_train = get_residual_sum_of_squares(reg_3,
                                          train_data.loc[:, model_3_features],
                                          train_data.loc[:, 'price'].values)

print('RSS of model 1: %E' % rss_1_train)
print('RSS of model 2: %E' % rss_2_train)
print('RSS of model 3: %E' % rss_3_train)

RSS of model 1: 9.678800E+14
RSS of model 2: 9.584196E+14
RSS of model 3: 9.034365E+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Answer: model 3 has the lowest RSS on training data. Yes, this is what I expected. RSS should decrease when more variables are added in model.

Now compute the RSS on on TEST data for each of the three models.

In [20]:
# Compute the RSS on TESTING data for each of the three models and record the values:
rss_1 = get_residual_sum_of_squares(reg_1,
                                    test_data.loc[:, model_1_features],
                                    test_data.loc[:, 'price'].values)

rss_2 = get_residual_sum_of_squares(reg_2,
                                    test_data.loc[:, model_2_features],
                                    test_data.loc[:, 'price'].values)

rss_3 = get_residual_sum_of_squares(reg_3,
                                    test_data.loc[:, model_3_features],
                                    test_data.loc[:, 'price'].values)

print('RSS of model 1: %E' % rss_1)
print('RSS of model 2: %E' % rss_2)
print('RSS of model 3: %E' % rss_3)

RSS of model 1: 2.255005E+14
RSS of model 2: 2.233775E+14
RSS of model 3: 2.592363E+14


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

Answer: model 2 has the lowest RSS on TESTING data.